In [44]:
# Azure OpenAI 호출 함수

import os
from dotenv import load_dotenv
import openai
import requests
import json
import textwrap

load_dotenv()

openai.api_type = "azure"
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_ENDPOINT")
openai.api_version = os.getenv("OPENAI_API_VERSION")


# 최근 대화 히스토리 저장용 (최대 20개 유지)
chat_history = []


def call_azure_openai(messages_or_text, model_deployment="gpt-4o", temperature=0.8, max_tokens=4000):
    global chat_history  # 외부 리스트 참조

    if isinstance(messages_or_text, str):
        user_msg = {"role": "user", "content": messages_or_text}

        # 시스템 메시지는 맨 처음에만 포함
        if not chat_history:
            system_msg = {
                "role": "system",
                "content": """너는 전문 퍼스널 트레이너 AI야.
사용자의 인바디 정보, 운동 경험, 체지방률, 목표 등을 바탕으로 
운동 루틴과 식단, 생활습관 개선에 대해 맞춤형으로 조언해줘.
- 운동 종류는 초보자도 이해할 수 있도록 설명해
- 필요한 경우 횟수/세트/시간도 구체적으로 제안해
- 질문이 모호하더라도 친절하게 유도 질문으로 도와줘
- 운동 안전 수칙도 적절히 포함해
- 필요하면 스트레칭, 자세 교정도 알려줘
- 사용자의 수면 정도, 스트레스 여부 등 컨디션도 반영을 해줘
- 지나치게 짧거나 과도하게 길게 답하지 말고, 핵심만 명확하게 전달해
친절하고 자신감 있는 말투로 응답해줘."""
            }
            chat_history.append(system_msg)

        # 사용자 메시지 추가
        chat_history.append(user_msg)

        # 최근 20개로 제한 (시스템 포함 최대 21개)
        chat_history = chat_history[-21:]

        messages = chat_history
    else:
        messages = messages_or_text  # 직접 리스트 입력된 경우 그대로 사용

    response = openai.ChatCompletion.create(
        engine=model_deployment,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )

    output = response["choices"][0]["message"]["content"]

    # GPT 응답도 히스토리에 추가
    chat_history.append({"role": "assistant", "content": output})
    chat_history = chat_history[-21:]

    # 줄바꿈 문자 파싱 처리
    try:
        output = json.loads(f'"{output}"')
    except json.JSONDecodeError:
        pass

    return output




# 비전 ocr 기능
VISION_KEY = os.getenv("AZURE_VISION_KEY")
VISION_ENDPOINT = os.getenv("AZURE_VISION_ENDPOINT")

def extract_text_from_image(image_path: str) -> str:
    url = f"{VISION_ENDPOINT}/computervision/imageanalysis:analyze?api-version=2023-10-01"
    headers = {
        "Ocp-Apim-Subscription-Key": VISION_KEY,
        "Content-Type": "application/octet-stream"
    }
    params = {
        "features": "read",
        "language": "ko"
    }

    with open(image_path, "rb") as image_data:
        response = requests.post(url, headers=headers, params=params, data=image_data)

    if response.status_code != 200:
        print(f"[OCR 실패] 상태 코드: {response.status_code}")
        return ""

    result = response.json()
    lines = []
    try:
        for block in result["readResult"]["blocks"]:
            for line in block["lines"]:
                lines.append(line["text"])
    except:
        return "[텍스트 추출 실패]"

    return "\n".join(lines)


# 프롬프트 템플릿 빌더

def build_prompt_from_inbody_text(ocr_text: str) -> str:
    return f"""
너는 전문 퍼스널 트레이너 AI야. 아래 인바디 검사 결과를 바탕으로 체성분을 분석하고, 실제 피트니스 센터 트레이너가 회원을 1:1로 상담해주듯 구체적이고 실용적으로 응답해줘.

📄 [인바디 검사 결과]
{ocr_text}

📝 [요청 사항] 아래 7가지 주제를 포함해 작성해줘:

---



1. 📊 인바디 분석  
- 첨부한 이미지는 내 체성분 검사 결과입니다.  
- 이 결과를 바탕으로 체지방률, 근육량, 기초대사량 등 주요 수치를 분석해 주세요. 
- 부위별 근육 분석과 부위별 체지방 분석을 부위별로 정도(표준 이하, 표준, 표준 이상)를 정리해 주고 어느 부위가 체지방이 표준 이상이고 어느 부위가 근육량이 표준 이하 인지도 알려줘
- 분석 결과를 통해 내 현재 건강 상태와 필요한 부분(예: 근육 보강 필요, 체지방 감량 등)을 주제별로 평가하고, 추가로 근력 향상이나 관련 운동도 함께 제안해 주세요.

2. 🏋️ 운동 프로그램 추천  
- 체성분 분석 결과를 참고하여, 2개월간 주 5회 진행할 수 있는 맞춤형 운동 프로그램을 작성해 주세요.  
- 운동 프로그램은 웜업, 메인 운동, 쿨다운 단계로 구성하고, 각 단계에서 수행할 운동 종목, 세트 수, 반복 횟수, 휴식 시간, 그리고 주의사항 등을 구체적으로 설명해 주세요.  
- 또한, 운동 목표(예: 근력 강화, 체지방 감량 등)에 따라 필요한 조정 사항도 포함해 주세요.

3. 🍱 식단 추천  
- 체성분 검사 결과와 운동 프로그램, 그리고 기존 식습관을 고려하여 이번 주 점심과 저녁 식단을 추천해 주세요.  
- 추천 메뉴는 적정 칼로리와 영양소의 균형을 맞추고, 직접 요리하지 않고 외부에서 쉽게 ‘구매’할 수 있는 메뉴(예: 도시락, 샐러드, 간편식 등)로 제안해 주세요.

4. 😴 수면 관리 및 개선  
- 최근 1주일간의 수면 패턴(취침 시간, 기상 시간, 수면 질 등)을 기반으로 내 수면의 질을 높일 수 있는 구체적인 방법을 제안해 주세요.  
- 취침 전 루틴, 최적의 수면 환경(조명, 소음, 온도 등), 그리고 생활 습관 개선 방안 등을 포함해 주세요.

5. 🧘‍♂️ 스트레스 관리 및 멘탈 케어  
- 최근 업무 및 일상 스트레스로 인해 정신적, 신체적 피로가 누적되고 있습니다.  
- 2개월 동안 실천할 수 있는 스트레스 관리 및 멘탈 케어 프로그램을 구체적으로 작성해 주세요.  
- 일상에서 실천 가능한 명상, 호흡 운동, 짧은 휴식법 등 구체적인 실행 계획을 포함해 주세요.

6. 🛡️ 부상 예방 및 재활 운동 추천  
- 현재의 운동 루틴을 고려할 때 부상 위험성이 우려됩니다.  
- 올바른 스트레칭, 준비 운동과 쿨다운, 그리고 부상 발생 시 초기 대응 및 재활 운동 방법에 대한 구체적인 가이드를 작성해 주세요.  
- 운동 중 올바른 자세와 부상 예방을 위한 주의사항도 포함해 주세요.

7. 💊 보충제 및 영양제 추천  
- 내 체성분 검사 결과와 현재 운동 및 식단 상황을 고려하여, 보충제나 영양제 섭취가 필요한지 평가해 주세요.  
- 만약 필요하다면 추천할 보충제 종류(예: 단백질 보충제, 비타민, 오메가-3 등)와 적절한 섭취 방법, 복용 시 주의해야 할 사항들을 구체적으로 설명해 주세요.

📚 [응답 형식 가이드]
- 각 항목은 숫자 또는 제목으로 구분해서 작성
- 문단은 항목마다 명확히 구분
- 표 형식은 마크다운으로
- 설명하면서도 트레이너다운 자연스러운 말투로 안내
"""


from IPython.display import display
import ipywidgets as widgets

upload = widgets.FileUpload(accept='image/*', multiple=False)
display(upload)

# 🔄 업로드된 이미지 저장 및 OCR 처리
if upload.value:
    uploaded_file = next(iter(upload.value.values()))
    filename = uploaded_file['metadata']['name']
    image_path = f"images/{filename}"  # 저장 경로

    # 이미지 저장
    os.makedirs("images", exist_ok=True)
    with open(image_path, "wb") as f:
        f.write(uploaded_file["content"])

    print(f"📁 이미지 저장 완료: {image_path}")

    # 1. OCR
    ocr_text = extract_text_from_image(image_path)
    print("📄 OCR 결과:\n", ocr_text)

    # 2. 템플릿 빌더로 GPT 프롬프트 구성
    prompt = build_prompt_from_inbody_text(ocr_text)

    # 3. GPT 호출
    response = call_azure_openai(prompt)
    print("🏋️‍♂️ GymPT 분석 결과:\n", response)
else:
    print("⚠️ 파일이 업로드되지 않았습니다.")





FileUpload(value=(), accept='image/*', description='Upload')

⚠️ 파일이 업로드되지 않았습니다.


In [45]:
# gradio_inbody_chat.py

import gradio as gr
import os
from dotenv import load_dotenv
from PIL import Image
import tempfile

#키워드에 따른 검색
# 키워드 → 섹션번호 매핑
KEYWORD_TOPIC_MAP = {
    "인바디": 1, "체성분": 1,
    "운동": 2, "운동 프로그램": 2,
    "식단": 3, "식사": 3,
    "수면": 4, "잠": 4,
    "스트레스": 5, "멘탈": 5,
    "부상": 6, "재활": 6,
    "보충제": 7, "영양제": 7,
}

SECTION_TEMPLATE = {
    1: """📊 1. 인바디 분석
- 첨부한 이미지는 내 체성분 검사 결과입니다.  
- 이 결과를 바탕으로 체지방률, 근육량, 기초대사량 등 주요 수치를 분석해 주세요.  
- 부위별 근육 분석과 부위별 체지방 분석을 부위별로 정도(표준 이하, 표준, 표준 이상)를 정리해 주고 어느 부위가 체지방이 표준 이상이고 어느 부위가 근육량이 표준 이하 인지도 알려줘  
- 분석 결과를 통해 내 현재 건강 상태와 필요한 부분(예: 근육 보강 필요, 체지방 감량 등)을 주제별로 평가하고, 추가로 근력 향상이나 관련 운동도 함께 제안해 주세요.
""",
    2: """🏋️ 2. 운동 프로그램 추천  
- 체성분 분석 결과를 참고하여, 2개월간 주 5회 진행할 수 있는 맞춤형 운동 프로그램을 작성해 주세요.  
- 운동 프로그램은 웜업, 메인 운동, 쿨다운 단계로 구성하고, 각 단계에서 수행할 운동 종목, 세트 수, 반복 횟수, 휴식 시간, 그리고 주의사항 등을 구체적으로 설명해 주세요.  
- 또한, 운동 목표(예: 근력 강화, 체지방 감량 등)에 따라 필요한 조정 사항도 포함해 주세요.
""",
    3: """🍱 3. 식단 추천 
- 체성분 검사 결과와 운동 프로그램, 그리고 기존 식습관을 고려하여 이번 주 점심과 저녁 식단을 추천해 주세요.  
- 추천 메뉴는 적정 칼로리와 영양소의 균형을 맞추고, 직접 요리하지 않고 외부에서 쉽게 ‘구매’할 수 있는 메뉴(예: 도시락, 샐러드, 간편식 등)로 제안해 주세요.
""",
    4: """😴 4. 수면 관리 및 개선
- 최근 1주일간의 수면 패턴(취침 시간, 기상 시간, 수면 질 등)을 기반으로 내 수면의 질을 높일 수 있는 구체적인 방법을 제안해 주세요.  
- 취침 전 루틴, 최적의 수면 환경(조명, 소음, 온도 등), 그리고 생활 습관 개선 방안 등을 포함해 주세요.
""",
    5: """🧘‍♂️ 5. 스트레스 관리 및 멘탈 케어  
- 최근 업무 및 일상 스트레스로 인해 정신적, 신체적 피로가 누적되고 있습니다.  
- 2개월 동안 실천할 수 있는 스트레스 관리 및 멘탈 케어 프로그램을 구체적으로 작성해 주세요.  
- 일상에서 실천 가능한 명상, 호흡 운동, 짧은 휴식법 등 구체적인 실행 계획을 포함해 주세요.
""",
    6: """🛡️ 6. 부상 예방 및 재활 운동 추천  
- 현재의 운동 루틴을 고려할 때 부상 위험성이 우려됩니다.  
- 올바른 스트레칭, 준비 운동과 쿨다운, 그리고 부상 발생 시 초기 대응 및 재활 운동 방법에 대한 구체적인 가이드를 작성해 주세요.  
- 운동 중 올바른 자세와 부상 예방을 위한 주의사항도 포함해 주세요.
""",
    7: """💊 7. 보충제 및 영양제 추천  
- 내 체성분 검사 결과와 현재 운동 및 식단 상황을 고려하여, 보충제나 영양제 섭취가 필요한지 평가해 주세요.  
- 만약 필요하다면 추천할 보충제 종류(예: 단백질 보충제, 비타민, 오메가-3 등)와 적절한 섭취 방법, 복용 시 주의해야 할 사항들을 구체적으로 설명해 주세요.
"""
}


def extract_requested_sections(user_input: str) -> list[int]:
    matched = set()
    for keyword, section in KEYWORD_TOPIC_MAP.items():
        if keyword in user_input:
            matched.add(section)
    return sorted(list(matched))

def build_prompt_from_inbody_text(ocr_text: str, user_input: str) -> str:
    base = (
        "너는 전문 퍼스널 트레이너 AI야. 아래 인바디 검사 결과를 바탕으로 체성분을 분석하고, "
        "실제 피트니스 센터 트레이너처럼 실용적으로 조언해줘.\n\n"
        f"📄 [인바디 검사 결과]\n{ocr_text.strip()}\n\n"
    )
    requested = extract_requested_sections(user_input)

    # 입력 없으면 전체 출력
    if not user_input.strip():
        requested = list(range(1, 8))

    prompt_parts = [SECTION_TEMPLATE[i] for i in requested]
    return base + "\n".join(prompt_parts)



load_dotenv()

# 💬 상태 초기화
chat_history = []

def process_input(image, user_question):
    global chat_history

    # 🧠 1. 인바디 이미지가 있을 경우 → OCR + GPT 분석
    if image is not None:
        with tempfile.NamedTemporaryFile(delete=False, suffix=".png") as temp:
            image.save(temp.name)
            ocr_text = extract_text_from_image(temp.name)

        if not ocr_text.strip():
            return "❌ OCR 분석 실패. 더 선명한 인바디 사진을 업로드해주세요."
        
        prompt = build_prompt_from_inbody_text(ocr_text,user_question)
        response = call_azure_openai(prompt)
        return f"📄 인바디 분석 결과:\n\n{response}"
    
    # 💬 2. 일반 질문이 있을 경우 → GPT 대화 처리
    if user_question and user_question.strip():
        response = call_azure_openai(user_question)
        return f"🤖 GymPT:\n{response}"
    
    return "⚠️ 인바디 사진을 업로드하거나 질문을 입력해주세요."

# ✅ 새로운 함수 추가
def process_input_and_clear(image, user_question):
    result = process_input(image, user_question)
    return result, ""  # 질문창 비우기

#CSS

custom_css = """
body {
    background-color: #1C1C1C;
    font-family: 'Noto Sans KR', sans-serif;
    color: #E1E1E1;
}

.gr-markdown h2 {
    font-size: 1.8rem;
    color: #19CE60;  /* 톤다운된 네이버 녹색 */
    margin-bottom: 0.5rem;
}

.gr-markdown p {
    font-size: 1rem;
    color: #AAAAAA;
    margin-bottom: 1rem;
}

#upload_box, #question_box textarea, #output_box textarea {
    background-color: #2A2A2A;
    border: 1px solid #3C3C3C;
    color: #F2F2F2;
    border-radius: 1rem;
    padding: 10px;
    font-size: 1rem;
}

#question_box textarea {
    resize: none;
}

#output_box textarea {
    min-height: 300px;
    resize: vertical;
}

button {
    background-color: #19CE60 !important;  /* ✅ 톤다운된 초록 */
    color: #181818 !important;
    font-weight: bold;
    border-radius: 0.75rem !important;
    border: none !important;
    transition: background-color 0.2s ease;
}

button:hover {
    background-color: #12B250 !important;  /* ✅ 호버 시 더 어두운 녹색 */
}
"""


#Gadio
with gr.Blocks(css=custom_css) as demo:
    gr.Markdown("## 🏋️‍♂️ GymPT: 퍼스널 트레이너 AI", elem_id="title")
    gr.Markdown("**인바디 사진을 업로드하거나 운동/식단 관련 질문을 해보세요.**")

    with gr.Row():
        image_input = gr.Image(type="pil", label="📸 인바디 사진", elem_id="upload_box")
        user_input = gr.Textbox(lines=1, placeholder="운동이나 식단 관련 질문을 입력하세요", label="💬 질문", elem_id="question_box")

    submit_btn = gr.Button("🚀 분석 요청")
    output_area = gr.Textbox(label="🔍 분석 결과 또는 답변", lines=15, elem_id="output_box")


 


    # 제출 동작 연결
    user_input.submit(
        fn=process_input_and_clear,
        inputs=[image_input, user_input],
        outputs=[output_area, user_input] 
    )

    submit_btn.click(fn=process_input, inputs=[image_input, user_input], outputs=output_area)


# 실행
if __name__ == "__main__":
    demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7891
* Running on public URL: https://57d415e117551ce6a6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [7]:
pip install openai==0.28

     ---------------------------------------- 0.0/76.3 kB ? eta -:--:--
     ---------------------------------------- 76.3/76.3 kB 4.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/76.5 kB ? eta -:--:--
   ---------------------------------------- 76.5/76.5 kB 4.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/452.3 kB ? eta -:--:--
   --------------------------------------  450.6/452.3 kB 13.8 MB/s eta 0:00:01
   ---------------------------------------- 452.3/452.3 kB 9.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/44.0 kB ? eta -:--:--
   ---------------------------------------- 44.0/44.0 kB 2.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/45.9 kB ? eta -:--:--
   ---------------------------------------- 45.9/45.9 kB 2.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
   ---------------------------------------- 41.5/41.5 kB ? eta 0:00:00
   -----------------------------


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
